Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [65]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


In [66]:
def test_bond_energies(htf, is_old=True, is_solvated=False):
    # Get harmonic bond force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_bond_force = system.getForce(0) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom bond force and hybrid positions
    bond_force_index = 1 if is_solvated else 0
    hybrid_system = htf.hybrid_system
    custom_bond_force = hybrid_system.getForce(bond_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_bond_force.getNumGlobalParameters()):
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new bonds in the custom bond force
    hybrid_to_bond_indices = htf._hybrid_to_new_bond_indices if is_old else htf._hybrid_to_old_bond_indices
    for hybrid_idx, idx in hybrid_to_bond_indices.items():
        p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        index_to_zero = -3 if is_old else -1
        hybrid_params[index_to_zero] *= 0
        custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_bond_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[0][1]], [components_hybrid[0][1]])
    
    print("Success! Custom bond force and standard bond force energies are equal!")

# Alanine dipeptide in vacuum

In [67]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


KeyboardInterrupt: 

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [68]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [69]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [70]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [71]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.881703491608262), ('HarmonicAngleForce', 8.64103997470652), ('PeriodicTorsionForce', 27.239469309657512), ('NonbondedForce', 517.8644869513105), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.8817034916082622), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [34]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.1965396232237354.
INFO:geometry:	reduced angle potential = 0.018416399305230712.
INFO:geometry:	reduced angle potential = 2.2729798870121365.
INFO:geometry:	reduced angle potential = 0.06001985692343648.


conducting subsequent work with the following platform: Reference


INFO:geometry:	reduced angle potential = 1.3189241736891264.
INFO:geometry:	reduced angle potential = 0.11093483696516669.
INFO:geometry:	reduced angle potential = 0.7622905653586042.
INFO:geometry:	reduced angle potential = 0.917039413411143.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 0.3517324280898812.
INFO:geometry:	reduced angle potential = 1.2915588460963948e-10.
INFO:geometry:	reduced angle potential = 3.205832446488702e-13.
INFO:geometry:	reduced angle potential = 7.39096069988752e-11.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA


KeyboardInterrupt: 

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [72]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [73]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [74]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [75]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [76]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.271134613450782), ('HarmonicAngleForce', 4.2906690195038655), ('PeriodicTorsionForce', 27.033057843000552), ('NonbondedForce', 166.33535360539238), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.271134613450782), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# Alanine dipeptide in solvent

In [26]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...


making topology proposal


INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_generator:{9: 7}
INFO:geometry:propose: performing forward proposal
INFO:geom

generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467829390215
INFO:geometry:magnitude of difference in the energies: 1.8634386549365445e-06
INFO:geometry:Final logp_proposal: -27149.66583273809


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]


### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [77]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [78]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -927.788883809082), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [79]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [80]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.268805346044796), ('HarmonicAngleForce', 6.544633801269215), ('PeriodicTorsionForce', 27.180005749631242), ('NonbondedForce', -891.8705506222258), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.268805346044804), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [29]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}


making topology proposal


INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_generator:{9: 7}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 13, 15, 19, 16, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direct

generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.9900002084555819, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 1: (19, 18, 10, 8, [1, Quantity(value=-0.07858019216845058, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 2: (19, 18, 10, 14, [1, Quantity(value=2.118586571250296, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 3: (19, 18, 10, 13, [1, Quantity(value=-2.149781203448759, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 4: (8, 10, 14, 17, [1, Quantity(value=-2.1249532223845904, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 8]), 5: (8, 10, 14, 16, [1, Quantity(value=2.0543043017157916, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 7]), 6: (8, 10, 14, 15, [1, Quantity(value=-0.02223392579985184, unit=radian), Quantity(value=1

...

INFO:geometry:setting atoms_with_positions context old positions
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



conducting subsequent work with the following platform: CUDA


INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-b0a546aa2d91>", line 3, in <module>
    htf = generate_dipeptide_top_pos_sys(atp.topology,
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/tests/test_topology_proposal.py", line 283, in generate_dipeptide_top_pos_sys
    logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta, validate_energy_bookkeeping=validate_energy_bookkeeping)
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/rjmc/geometry.py", line 287, in logp_reverse
    logp_proposal, new_positions, rjmc_info, atoms_with_positions_reduced_potential, final_context_reduced_potential, neglected_angle

TypeError: object of type 'NoneType' has no len()

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [81]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [82]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [83]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -1041.13577274055), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [84]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [85]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.4684578607702736), ('HarmonicAngleForce', 4.80952399451105), ('PeriodicTorsionForce', 27.201127706572517), ('NonbondedForce', -1022.2262834587337), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.4684578607702563), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# CDK2 transformation in solvent

In [6]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [2]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-04-29 18:17:44,621:(0.00s):root:	Detecting phases...
2021-04-29 18:17:44,623:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-04-29 18:17:44,624:(0.00s):root:No constraints will be removed
2021-04-29 18:17:44,626:(0.00s):root:No spectators
2021-04-29 18:17:44,628:(0.00s):root:			run_type is not specified; default to None
2021-04-29 18:17:44,630:(0.00s):root:	Detecting fe_type...
2021-04-29 18:17:44,632:(0.00s):root:		fe_type: repex
2021-04-29 18:17:44,633:(0.00s):root:			offline-freq not specified: default to 10.
2021-04-29 18:17:44,635:(0.00s):root:	'neglect_angles' detected: False.
2021-04-29 18:17:44,637:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-04-29 18:17:44,638:(0.00s):root:	Creating 'output'...
2021-04-29 18:17:44,641:(0.00s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-04-29 18:17:44,643:(0.00s):ro

2021-04-29 18:17:49,917:(0.00s):geometry:propose: unique new atoms detected; proceeding to _logp_propose...
2021-04-29 18:17:49,918:(0.00s):geometry:Conducting forward proposal...
2021-04-29 18:17:49,920:(0.00s):geometry:Computing proposal order with NetworkX...
2021-04-29 18:17:49,930:(0.01s):geometry:number of atoms to be placed: 1
2021-04-29 18:17:49,931:(0.00s):geometry:Atom index proposal order is [2190]
2021-04-29 18:17:49,933:(0.00s):geometry:omitted_bonds: []
2021-04-29 18:17:49,934:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-04-29 18:17:50,094:(0.16s):geometry:creating growth system...
2021-04-29 18:17:50,134:(0.04s):geometry:	creating bond force...
2021-04-29 18:17:50,136:(0.00s):geometry:	there are 27 bonds in reference force.
2021-04-29 18:17:50,138:(0.00s):geometry:	creating angle force...
2021-04-29 18:17:50,141:(0.00s):geometry:	there are 85 angles in reference force.
2021-04-29 18:17

conducting subsequent work with the following platform: CUDA


2021-04-29 18:17:50,942:(0.72s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-04-29 18:17:58,053:(1.23s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-04-29 18:17:59,558:(1.50s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-04-29 18:17:59,560:(0.00s):geometry:final reduced energy 43.36216011253515
2021-04-29 18:17:59,562:(0.00s):geometry:sum of energies: 43.36215903557945
2021-04-29 18:17:59,563:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-04-29 18:17:59,564:(0.00s):geometry:Final logp_proposal: 11.821616720480304


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-04-29 18:17:59,755:(0.19s):root:	writing pickle output...
2021-04-29 18:17:59,869:(0.11s):root:	successfully dumped pickle.
2021-04-29 18:17:59,873:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-04-29 18:17:59,875:(0.00s):root:	writing atom_mapping.png
2021-04-29 18:18:00,149:(0.27s):root:	 steps per move application: 50
2021-04-29 18:18:00,153:(0.00s):root:	trajectory prefix: out
2021-04-29 18:18:00,155:(0.00s):root:	atom selection detected: not water
2021-04-29 18:18:00,157:(0.00s):root:	no nonequilibrium detected.
2021-04-29 18:18:00,159:(0.00s):root:	cataloging HybridTopologyFactories...
2021-04-29 18:18:00,161:(0.00s):root:		phase: solvent:
2021-04-29 18:18:00,163:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-04-29 18:18:00,165:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-04-29 18:18:00,167:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [29]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [11]:
htf = setup_dict['hybrid_topology_factories']['solvent']

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [86]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [87]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1700.223999491089), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [88]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [89]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.16743219736821), ('PeriodicTorsionForce', 15.08569135447898), ('NonbondedForce', 1717.5107626234537), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [6]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [55]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-04-29 18:27:10,995:(4.85s):root:	Detecting phases...
2021-04-29 18:27:10,998:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-04-29 18:27:11,001:(0.00s):root:No constraints will be removed
2021-04-29 18:27:11,004:(0.00s):root:No spectators
2021-04-29 18:27:11,006:(0.00s):root:			run_type is not specified; default to None
2021-04-29 18:27:11,009:(0.00s):root:	Detecting fe_type...
2021-04-29 18:27:11,011:(0.00s):root:		fe_type: repex
2021-04-29 18:27:11,013:(0.00s):root:			offline-freq not specified: default to 10.
2021-04-29 18:27:11,014:(0.00s):root:	'neglect_angles' detected: False.
2021-04-29 18:27:11,015:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-04-29 18:27:11,019:(0.00s):root:	Creating 'output'...
2021-04-29 18:27:11,030:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-04-29 18:27:11,031:(0.00s):ro

2021-04-29 18:27:16,723:(0.00s):geometry:Conducting forward proposal...
2021-04-29 18:27:16,725:(0.00s):geometry:Computing proposal order with NetworkX...
2021-04-29 18:27:16,742:(0.02s):geometry:number of atoms to be placed: 1
2021-04-29 18:27:16,744:(0.00s):geometry:Atom index proposal order is [2190]
2021-04-29 18:27:16,746:(0.00s):geometry:omitted_bonds: []
2021-04-29 18:27:16,748:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-04-29 18:27:17,061:(0.31s):geometry:creating growth system...
2021-04-29 18:27:17,141:(0.08s):geometry:	creating bond force...
2021-04-29 18:27:17,145:(0.00s):geometry:	there are 27 bonds in reference force.
2021-04-29 18:27:17,147:(0.00s):geometry:	creating angle force...
2021-04-29 18:27:17,149:(0.00s):geometry:	there are 85 angles in reference force.
2021-04-29 18:27:17,153:(0.00s):geometry:	creating torsion force...
2021-04-29 18:27:17,155:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-04-29 18:27:18,191:(0.92s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-04-29 18:27:28,199:(1.84s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-04-29 18:27:30,469:(2.27s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-04-29 18:27:30,472:(0.00s):geometry:final reduced energy 43.36216011253515
2021-04-29 18:27:30,475:(0.00s):geometry:sum of energies: 43.36215903557945
2021-04-29 18:27:30,477:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-04-29 18:27:30,480:(0.00s):geometry:Final logp_proposal: 11.56594796641107


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-04-29 18:27:30,778:(0.30s):root:	writing pickle output...
2021-04-29 18:27:30,998:(0.22s):root:	successfully dumped pickle.
2021-04-29 18:27:31,005:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-04-29 18:27:31,008:(0.00s):root:	writing atom_mapping.png
2021-04-29 18:27:31,092:(0.08s):root:	 steps per move application: 50
2021-04-29 18:27:31,096:(0.00s):root:	trajectory prefix: out
2021-04-29 18:27:31,099:(0.00s):root:	atom selection detected: not water
2021-04-29 18:27:31,101:(0.00s):root:	no nonequilibrium detected.
2021-04-29 18:27:31,104:(0.00s):root:	cataloging HybridTopologyFactories...
2021-04-29 18:27:31,106:(0.00s):root:		phase: solvent:
2021-04-29 18:27:31,109:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-04-29 18:27:31,111:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-04-29 18:27:31,114:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [56]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [58]:
htf = setup_dict['hybrid_topology_factories']['solvent']

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [90]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [91]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [92]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1766.9022231456272), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [93]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [94]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.31629908967661), ('PeriodicTorsionForce', 15.07821281507567), ('NonbondedForce', 1783.5471715609788), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [ ]:
# TO DO: actually test the scaling of the energies matches